In [1]:
import pyrealsense2 as rs
import numpy as np
import cv2
import rtde_control
import easyocr
import matplotlib.pyplot as plt
import openpyxl
import time
#import robotiq_gripper
import points2 as p
from statistics import mode
import re

import destination as pd
dest=pd.dest_data()

import rtde_control
import rtde_receive
rtde_r = rtde_receive.RTDEReceiveInterface("169.254.37.182")
rtde_c = rtde_control.RTDEControlInterface("169.254.37.182")

#Gripper
ip = "169.254.37.182"
import robotiq_gripper
def log_info(gripper):
    print(f"Pos: {str(gripper.get_current_position()): >3}  "
          f"Open: {gripper.is_open(): <2}  "
          f"Closed: {gripper.is_closed(): <2}  ")
print("Creating gripper...")
gripper = robotiq_gripper.RobotiqGripper()
print("Connecting to gripper...")
gripper.connect(ip, 63352)
print("Activating gripper...")
gripper.activate()

# Parameters
velocity = 0.5
acceleration = 0.5
dt = 1.0/500  # 2ms
lookahead_time = 0.1
gain = 300

#Jig Location Dictionary
jig_coord={1:[-0.36785,-0.68958,0.11495,2.232,2.201,-0.005],2:[-0.30191, -0.68953,0.11495,2.232,2.201,-0.005],3:[-0.24006, -0.68953,0.11495,2.232,2.201,-0.005],4:[-0.18001, -0.68951,0.11495,2.232,2.201,-0.005],5:[-0.11999, -0.68953,0.11495,2.232,2.201,-0.005],6:[-0.05998, -0.68979,0.11495,2.232,2.201,-0.005],7:[-0.00003, -0.69093,0.11495,2.232,2.201,-0.005],8:[0.05998, -0.69145,0.11495,2.232,2.201,-0.005],9:[0.12496,-0.68984,0.11495,2.232,2.201,-0.005]}

## Pick and Place 12
rtde_c.moveL([0.27967, -0.58801, 0.11500, 2.232, 2.201, -0.005], 0.5, 0.3)
rtde_c.moveL([0.27973, -0.58695, -0.04, 2.232, 2.201, -0.005], 0.5, 0.3)
gripper.move_and_wait_for_pos(224, 255, 255)
rtde_c.moveL([0.27971, -0.58834, 0.11499, 2.223, 2.190, -0.018], 0.5, 0.3)
rtde_c.moveL([0.05576, -0.45815, 0.04500, 2.239, 2.201, -0.005], 0.5, 0.3)
rtde_c.moveL([0.05576, -0.45745, 0.02513, 2.239, 2.201, -0.005], 0.5, 0.3)
rtde_c.moveL([0.11360, -0.45744, 0.02512, 2.239, 2.201, -0.005], 0.5, 0.3)
gripper.move_and_wait_for_pos(0, 255, 255)
rtde_c.moveL([0.09360, -0.45744, 0.15, 2.239, 2.201, -0.005], 0.5, 0.3)

result_f={}
depth_dict={}

#Capturing position
rtde_c.moveL([-0.00772, -0.64201, 0.19098, 0.032, -3.154, -0.017], 0.5, 0.3)
time.sleep(1)
num=p.point()
print(num)

def depth_filter(win_data):
    x1,y1,x2,y2=win_data[0]-win_data[2]/2,win_data[1]-win_data[3]/2,win_data[0]+3*win_data[2]/2,win_data[1]+3*win_data[3]/2
    for i in range(int(y1),int(y2)):
        for j in range(int(x1),int(x2)):
            d=aligned_depth_frame.get_distance(int(j),int(i))
            if d!=0:
                k=[j,i]
                depth_dict[f'{k}']=round(d, 2)
    depth_list=depth_dict.values()
    filtered_depth=mode(depth_list)
    print(filtered_depth)
    corr_pix=list(depth_dict.keys())[list(depth_dict.values()).index(filtered_depth)] #get corresponding pixel of the distance value
    print(corr_pix)
    print(type(corr_pix))
    #print(corr_pix) #A string here
    corr_pix=re.split(', ',corr_pix)
    pix_corr=[]
    for num in corr_pix:
        num=re.sub(r"[\([{})\]]", "", num)
        num=int(num)
        pix_corr.append(num)
    
    print(pix_corr)
    print(type(pix_corr))
    a,b=pix_corr[0],pix_corr[1]
    depth=aligned_depth_frame.get_distance(int(a),int(b))
    return depth
def distance(x,y):
    #cv2.circle(capture,(x,y),2,(128,0,128),-1)
    #print((x,y))
    d=depth_frame.get_distance(int(x),int(y))
    print(d)
    if (d==0.0) :
        print('No depth info, Using an window')
        depth=depth_filter([int(x),int(y),10,10])
        x_w, y_w, z_w = convert_depth_to_phys_coord_using_realsense(int(x),int(y), depth, camera_info)
        print('Camera Points are:',x_w, y_w, z_w)
        p1=f'{x_w} {y_w} {z_w}'
        p = [float(value) for value in p1.split(' ')]
        p.append(1.0)
        dst_p=np.matmul(k,p)
        dst_p=dst_p.tolist()
        print(dst_p)
        #rtde_c.moveL([dst_p[0]/1000.0, dst_p[1]/1000.0, dst_p[2]/1000.0, 0.032, -3.154, -0.017], 0.5, 0.3)
        #time.sleep(5)
        #dst_p=(dst_p[0])[:-1]
        #print(dst_p)
        obj1=dst_p[0]
        list=[]
        for i in range(1,10):
            list.append(abs(obj1-(jig_coord[i][0]*1000)))
        obj1abs=list.index(min(list))+1
        print(obj1abs)
        rtde_c.moveL(jig_coord[obj1abs], 0.5, 0.3)
        del list
        del p1
        del p
    else:
        x_w, y_w, z_w = convert_depth_to_phys_coord_using_realsense(int(x),int(y), d, camera_info)
        print('Camera Points are:',x_w, y_w, z_w)
        p1=f'{x_w} {y_w} {z_w}'
        p = [float(value) for value in p1.split(' ')]
        p.append(1.0)
        dst_p=np.matmul(k,p)
        dst_p=dst_p.tolist()
        print(dst_p)
        #rtde_c.moveL([dst_p[0]/1000.0, dst_p[1]/1000.0, dst_p[2]/1000.0, 0.032, -3.154, -0.017], 0.5, 0.3)
        #time.sleep(5)
        #dst_p=(dst_p[0])[:-1]
        #print(dst_p)
        obj1=dst_p[0]
        list=[]
        for i in range(1,10):
            list.append(abs(obj1-(jig_coord[i][0]*1000)))
        obj1abs=list.index(min(list))+1
        print(obj1abs)
        rtde_c.moveL(jig_coord[obj1abs], 0.5, 0.3)
        del list
        del p1
        del p

def convert_depth_to_phys_coord_using_realsense(x, y, depth, cameraInfo):
    _intrinsics = rs.intrinsics()
    _intrinsics.width = cameraInfo.width
    _intrinsics.height = cameraInfo.height
    _intrinsics.ppx = cameraInfo.ppx
    _intrinsics.ppy = cameraInfo.ppy
    _intrinsics.fx = cameraInfo.fx
    _intrinsics.fy = cameraInfo.fy
    # _intrinsics.model = cameraInfo.distortion_model
    _intrinsics.model  = rs.distortion.none
    _intrinsics.coeffs = [i for i in cameraInfo.coeffs]
    result = rs.rs2_deproject_pixel_to_point(_intrinsics, [x, y], depth)
    # result[0]: right, result[1]: down, result[2]: forward
    return result[0], -result[1], -result[2]
"""
# Setup the pipeline
pipe = rs.pipeline()
cfg = rs.config()
cfg.enable_stream(rs.stream.depth, 1280, 720, rs.format.z16, 30)
cfg.enable_stream(rs.stream.color, 1280, 720, rs.format.bgr8, 30)
profile = pipe.start(cfg)
"""

k= [[-9.79365051e+02,6.22173726e-01,-1.81205747e+02,-1.09894855e+01],[2.63002626e+00,-9.88050938e+02,-2.41157019e+01,-6.93151121e+02],[1.52250424e+02,7.62100894e+01,-4.30734272e+02,4.48864321e+01],[0.00000000e+00,0.00000000e+00,0.00000000e+00,1.00000000e+00]]


# Create a pipeline
pipeline = rs.pipeline()

# Create a config and configure the pipeline to stream
config = rs.config()
pipeline_wrapper = rs.pipeline_wrapper(pipeline)
pipeline_profile = config.resolve(pipeline_wrapper)
device = pipeline_profile.get_device()
device_product_line = str(device.get_info(rs.camera_info.product_line))
print(device_product_line)
found_rgb = False
for s in device.sensors:
    if s.get_info(rs.camera_info.name) == 'RGB Camera':
        found_rgb = True
        print("There is a depth camera with color sensor")
        break
if not found_rgb:
    print("The demo requires Depth camera with Color sensor")
    exit(0)
config.enable_stream(rs.stream.depth, 848,480, rs.format.z16, 30)
config.enable_stream(rs.stream.color, 848,480, rs.format.bgr8, 30)
profile = pipeline.start(config)


# Setup the 'High Accuracy'-mode
depth_sensor = profile.get_device().first_depth_sensor()
depth_scale = depth_sensor.get_depth_scale()
print("Depth Scale is: " , depth_scale)
clipping_distance_in_meters = 0.3 #1 meter
clipping_distance = clipping_distance_in_meters / depth_scale
print(clipping_distance)
preset_range = depth_sensor.get_option_range(rs.option.visual_preset)
for i in range(int(preset_range.max)):
    visulpreset = depth_sensor.get_option_value_description(rs.option.visual_preset,i)
    print('%02d: %s'%(i,visulpreset))
    if visulpreset == "High Accuracy":
        depth_sensor.set_option(rs.option.visual_preset, i)
# enable higher laser-power for better detection
depth_sensor.set_option(rs.option.laser_power, 180)
# lower the depth unit for better accuracy and shorter distance covered
depth_sensor.set_option(rs.option.depth_units, 0.0005)
align_to = rs.stream.color
align = rs.align(align_to)
# Skip first frames for auto-exposure to adjust
for x in range(5):
    pipeline.wait_for_frames()
t1=time.time()
try:
    while True:

        # Stores next frameset
        frames = pipeline.wait_for_frames()
        color_frame = frames.get_color_frame()
        depth_frame = frames.get_depth_frame()

        if color_frame:
            aligned_frames = align.process(frames)

            # Get aligned frames
            aligned_depth_frame = aligned_frames.get_depth_frame() # aligned_depth_frame is a 640x480 depth image
            color_frame = aligned_frames.get_color_frame()

            # Validate that both frames are valid
            if not aligned_depth_frame or not color_frame:
                continue

            depth_image = np.asanyarray(aligned_depth_frame.get_data())
            color_image = np.asanyarray(color_frame.get_data())

            # Remove background - Set pixels further than clipping_distance to grey
            black_color = 0
            depth_image_3d = np.dstack((depth_image,depth_image,depth_image)) #depth image is 1 channel, color is 3 channels
            bg_removed = np.where((depth_image_3d > clipping_distance) | (depth_image_3d <= 0), black_color, color_image)

            # Render images:
            #   depth align to color on left
            #   depth on right
            #depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.03), cv2.COLORMAP_JET)
            #images = np.hstack((bg_removed, depth_colormap))
            camera_info = aligned_depth_frame.profile.as_video_stream_profile().intrinsics
            t2=time.time()
            if (t2-t1)>3:
                cv2.imwrite('captured2.jpg',bg_removed)
                # cv2.imshow('captured',bg_removed)
                # cv2.waitKey(0)
                # # cv2.destroyAllWindows()
                break
                
finally:
    numbers=list(num.keys())
    z=14
    #rtde_c.moveL([-0.16780, -0.70145, 0.160, 0.036, -3.154, -0.017], 0.5, 0.3) #Capturing position
    time.sleep(1)
    

    
    for j in range(9):
        if f'{z}' in numbers:
            print(f'{z}yes')
            val=num[f'{z}']
            distance(val[0],val[1])
            time.sleep(1)
            
            ### Move downwards in Z axis
            actual_p=rtde_r.getActualTCPPose()
            actual_p1=actual_p[2]
            actual_p[2]=0.015
            rtde_c.moveL(actual_p)
            ### grip close to 80%
            gripper.move_and_wait_for_pos(224, 255, 255)
            ### lifts the part in Z axis
            actual_p=rtde_r.getActualTCPPose()
            actual_p[2]=actual_p1
            rtde_c.moveL(actual_p)
            ### Call the destination dictionary
            for k in range(4):
                rtde_c.moveL(dest[z][k], 0.5, 0.3)
            ### grippen open 
            gripper.move_and_wait_for_pos(0, 255, 255)
            ### move in Z axis
            actual_c=rtde_r.getActualTCPPose()
            actual_c[2]+=0.115
            rtde_c.moveL(actual_c)
            
            z+=2
        
    
    # Stop streaming
    rtde_c.disconnect()
    pipeline.stop()

Creating gripper...
Connecting to gripper...
Activating gripper...
Gripper auto-calibrated to [3, 248]


Using CPU. Note: This module is much faster with a GPU.


(480, 848, 3)
{'14': [158.0, 246.0], '16': [340.0, 259.0], '18': [517.0, 255.0], '20': [706.0, 256.0]}
D400
There is a depth camera with color sensor
Depth Scale is:  0.0005000000237487257
599.9999715015306
00: Custom
01: Default
02: Hand
03: High Accuracy
04: High Density
14yes
0.21050001680850983
Camera Points are: -0.09434901922941208 0.0018433876102790236 -0.21050001680850983
[119.55760622604967, -690.1442666003134, 121.3318101489173, 1.0]
9
16yes
0.21650001406669617
Camera Points are: -0.032042596489191055 -0.0027495943941175938 -0.21650001406669617


ValueError: matmul: Input operand 0 does not have enough dimensions (has 0, gufunc core with signature (n?,k),(k,m?)->(n?,m?) requires 1)